In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
import requests 
from IPython.core.display import display, HTML
from IPython.display import clear_output
import urllib
from urllib.request import urlopen
from urllib.parse import urlparse, urljoin
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'


with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url[:-1] for url in urls]

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_3362/588965286.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
## utils functions
from openai import OpenAI
head_prompt = """I'll give you an HTML element that's extracted from the web page of a company. I want you to estimate the likelihood """ +\
"""that the hyperlink in the HTML element is likely to lead to a page that's named something like "About the Team" or "About Us" or "Our Team" or """+\
"""it leads to a Bio page of any of the employees in the company where I can get information about the employee's name, """ +\
"""phone number, email address etc. \nProvide your estimate as a single integer number between 0 and 100, where 0 means very unlikely, """ +\
"""and 100 means absolutely certain. Do not include any other text or explanation in your response. \n""" +\
"""Here's the HTML tag: """
client = OpenAI()

def call_gpt(href, bios):
    prompt = head_prompt + str(href)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers only and only in integer numeric values"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.01,
        logprobs=False
    ) 
    content = completion.choices[0].message.dict()['content']
    estimate = int(float(content))
    print(f'estimate:{str(estimate):4s} {href} ')
    if estimate >= 40:
        bios.append({"URL":href, "estimate":estimate})

    return estimate


def get_href(root_url, atag):
    href = atag.get("href")
    if not urlparse(href).hostname:
        href = urljoin(root_url, href)
    if urlparse(href).hostname is not None and \
        not urlparse(href).hostname.endswith(urlparse(root_url).hostname) and not urlparse(root_url).hostname.endswith(urlparse(href).hostname):
        print(f'        hostname mismatch: {urlparse(href).hostname} != {urlparse(root_url).hostname}')
        return None
    return href
    
    
def get_atags(href):
    try:
        response = requests.get(href, timeout=8)
        soup = BeautifulSoup(response.content, 'html.parser')
        atags = soup.find_all("a") 
        if not response.ok:
            print(f'    *** RESPONSE NOT OK!! *** status_code: {response.status_code} ')
        return atags
    except Exception:
        print(f'    Exception at href: {href} {Exception}')
        return None


In [3]:
## BFS

from collections import deque
start = time.time()
bios = []
seen = set()
model = "gpt-4"

for iu, root_url in enumerate(urls[139:140]):
    print(f'\n{iu} or {len(urls)} main URL: {root_url}')
        
    queue = deque([(BeautifulSoup(f'<a href="{root_url}"/></a>', 'html.parser'), 0)]) 
    prev_depth = -1
    while queue and queue[0][1] < 3: 
        atag, depth = queue.popleft() 
        if prev_depth != depth:
            print(f'    ==> new depth:{depth}  len(queue):{len(queue)+1}')
            prev_depth = depth
        href = get_href(root_url, atag)
        if href is None or href in seen:
            continue
        seen.add(href)
        print(f'    depth:{depth} ', end=' ')

        estimate = call_gpt(href, bios)
        if depth > 1 and estimate == 0:
            continue
            
        atags = get_atags(href)
        if atags is None:
            continue
        for atag in atags:
            href = atag.get("href")
            if not urlparse(href).hostname:
                href = urljoin(root_url, href) 
            if urlparse(href).hostname is not None and \
                not urlparse(href).hostname.endswith(urlparse(root_url).hostname) and not urlparse(root_url).hostname.endswith(urlparse(href).hostname):
                continue
            queue.append((atag,depth+1))
        
    print(f'    len(bios) {len(bios)} {print_runtime(start, False)}')

# with open ('bios.pkl', 'wb') as f:
#     pickle.dump(bios, f)


0 or 1365 main URL: http://mabusinessadvisors.com/
    ==> new depth:0  len(queue):1
    depth:0  estimate:0    http://mabusinessadvisors.com/ 
    *** RESPONSE NOT OK!! *** status_code: 403 
    len(bios) 0 (0 min  0 sec)


In [5]:
root_url, href

('http://mabusinessadvisors.com/', 'http://mabusinessadvisors.com/')

In [22]:
# extract the text out of candidates "bios"
def extract_text_from_url(url, data, visited_urls):
    if url in visited_urls:
        print(f'url already in visited_urls: {url}')
        return None, None
    
    try:
        response = requests.get(url, timeout=8)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.text
    except Exception as e:
        stdout = f'{idx:5d}  url: {url}  I had an exception {e}'
        print(stdout)
        visited_urls.add(url)
        data[url] = stdout
        return None, None
    
    data[url] = text
    visited_urls.add(url)
    print(f'{idx:5d} {len(data[url]):6d} chars  {url} ')
    return text

data = dict()
visited_urls = set()
prev_num_chars = -1
for idx, bio in enumerate(bios):
    url = bio['URL']
    text = extract_text_from_url(url, data, visited_urls)
    num_chars = len(text)
    if prev_num_chars == num_chars:
        del data[url]
    prev_num_chars = num_chars
    time.sleep(.1)


    0   8792 chars  http://goldstarbbaz.com/OurTeam.aspx 
    1     52 chars  https://gottesman-company.com/about/ 
    2     52 chars  https://gottesman-company.com/team/ 
    3   1821 chars  https://www.greenbridgebrokers.com/copy-of-about 
    4   6986 chars  https://www.greenbridgebrokers.com/about 
    5   9855 chars  https://hardingbusinessfl.com/about/ 
    6   9855 chars  https://hardingbusinessfl.com/about/#who-about 
    7   9855 chars  https://hardingbusinessfl.com/about/#testimonials-about 
    8   5207 chars  https://www.harvestbusiness.com/about-1 
    9   3241 chars  https://harvestlandscapeconsulting.com/consulting/meet-the-harvesters/ 
   10   5385 chars  https://harvestlandscapeconsulting.com/consulting/meet-the-harvesters/harvester-bill-arman/ 
   11   5210 chars  https://harvestlandscapeconsulting.com/consulting/meet-the-harvesters/harvester-ed-laflamme/ 
   12   3145 chars  https://harvestlandscapeconsulting.com/consulting/meet-the-harvesters/harvester-cindy-code/ 

In [20]:
from openai import OpenAI
client = OpenAI()

results = []
start = time.time()
model = "gpt-4"


for idx, (url,  text) in enumerate(data.items()):
    s0 = time.time()
    text = data[url]

    prompt = """Here's some text I extracted from a webpage:\n""" + text +\
    """ \n Extract the following Named Entities from the above text: "First Name", "Middle Name", "Last Name", "Profession", """ +\
    """Company Name", "Phone Number", "Email", "Address" , "State", "Zipcode", "Bio", "Specialty Areas" """ +\
    """and  put them in a json file with the keys I specified. If the information is missing """ +\
    """leave the value of that key blank. """+\
    """Only return a JSON formatted text. Don't say anything else, don't write any python code. """ +\
    """If the given text contains more than one individual, create a separate json file and return a list of json files"""
    """If any of the values in the JSON file is a list of items, make sure you wrap the items in square brackets compatible with python syntax."""

    completion = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": "You are an assistant that extracts named entities from provided text, and returns an answer that's only comprised of " +\
         "json formatted text"},
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message.dict()['content']
    if ",}" in content:
        print('==> replacing ",}", "}" ' + f" {print_runtime(start, False)}")
        content = content.replace(',}', '}')
    if ",\n" in content:
        print(f'==> replacing ",\n" with ","' + f" {print_runtime(start, False)}")
        content = content.replace(",\n", ",")

    json_output = json.loads(content)
    if type(json_output) == list:
        for item in json_output:
            item['URL'] = url
            results.append(item)
    else:
        json_output['URL'] = url
        results.append(json_output)
    print(f'idx:{idx} finished. len(results):{len(results)} {print_runtime(s0, False)} \n')

print_runtime(start)

==> replacing ",
" with "," (0 min  3 sec)
idx:2 finished. len(results):1 (0 min  3 sec) 

Runtime: 0 min  3 sec


In [21]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('named_entities.csv')
df

,First Name,Middle Name,Last Name,Profession,Company Name,Phone Number,Email,Address,State,Zipcode,Bio,Specialty Areas,URL
0,,,,Consulting,Green Bridge Advisor,970.286.0052,christian@greenbridgebrokers.com,,,,,"Retail, Service, Agricultural, Professional, H...",https://www.greenbridgebrokers.com/copy-of-about
